In [3]:
import pandas as pd

df = pd.read_csv("./data/curData.csv")

In [11]:
df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)

In [12]:
for col in df.columns:
    print(col, ": ", df[col].dtype)

loan_amnt :  float64
term :  float64
int_rate :  float64
installment :  float64
purpose :  object
emp_length :  int64
home_ownership :  object
annual_inc :  float64
dti :  float64
inq_last_6mths :  float64
delinq_2yrs :  float64
open_acc :  float64
pub_rec :  float64
revol_bal :  float64
revol_util :  float64
total_acc :  float64
issue_d :  object
loan_risk :  int64
installment_to_income :  float64
fico_score :  float64
credit_age :  float64
issue_month :  object
fed_funds_rate :  float64
unemployment_rate :  float64
cpi :  float64
real_gdp :  float64
debt_service_ratio :  float64


In [21]:
import math
import numpy as np

ml_df = df.drop(["issue_d", "issue_month"], axis=1)
ml_df["loan_to_income"] = ml_df["loan_amnt"] / np.maximum(df["annual_inc"], 1)
ml_df["installment_to_loan"] = ml_df["installment"] / ml_df["loan_amnt"]
ml_df["annual_inc_log"] = np.log1p(ml_df["annual_inc"])
ml_df["rate_term"] = ml_df["int_rate"] * ml_df["term"]

ml_df["loan_risk"].unique()

array([0, 1])

In [ ]:
ml_features = list(ml_df.columns)
input_features = ml_features

['loan_amnt',
 'term',
 'int_rate',
 'installment',
 'purpose',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'dti',
 'inq_last_6mths',
 'delinq_2yrs',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'loan_risk',
 'installment_to_income',
 'fico_score',
 'credit_age',
 'fed_funds_rate',
 'unemployment_rate',
 'cpi',
 'real_gdp',
 'debt_service_ratio']

In [32]:
# Split into Train, Val, Test based on time (oldest -> recent)
ml_df["date"] = pd.to_datetime(df["issue_d"])
ml_df = ml_df.sort_values("date").reset_index(drop=True)

# Change as needed
train_frac = 0.8
val_frac = 0.1
test_frac = 0.1

n = len(ml_df)
train_n = int(train_frac * n)
val_n = int(val_frac * n)

train_df = df.iloc[:train_n] # First rows for training
val_df = df.iloc[train_n : train_n + val_n] # Next 0.1 for validation
test_df = df.iloc[train_n + val_n:] # Last for testing

In [35]:
print(len(train_df), len(val_df), len(test_df))
print(train_df["loan_risk"].value_counts(), val_df["loan_risk"].value_counts(), test_df["loan_risk"].value_counts())

1808534 226066 226068
loan_risk
0    1580265
1     228269
Name: count, dtype: int64 loan_risk
0    188951
1     37115
Name: count, dtype: int64 loan_risk
0    200625
1     25443
Name: count, dtype: int64


In [36]:
train_df.to_csv("./data/train.csv")
val_df.to_csv("./data/val.csv")
test_df.to_csv("./data/test.csv")